In [0]:
with cte_data (ranges, id) as
(
select transform(split(split_part(value, '\n\n', 1), '\n'), t -> (split_part(t, '-', 1)::bigint as b, split_part(t, '-', 2)::bigint as e))
     , explode(transform(split(split_part(value, '\n\n', 2), '\n'), t -> t::bigint))
  from read_files('/Volumes/waldar/fabien/aoc2025/aoc2025_05_input.txt', format => 'text', wholeText => true) as t (value)
/*from values ('3-5'   || '\n'
           ||  '10-14' || '\n'
           ||  '16-20' || '\n'
           ||  '12-18' || '\n'
           ||  '9-21'  || '\n'
           ||  ''      || '\n'
           ||  '1'     || '\n'
           ||  '5'     || '\n'
           ||  '8'     || '\n'
           ||  '11'    || '\n'
           ||  '17'    || '\n'
           ||  '32') as t (value) */
)
select count(*) filter(where array_size(filter(ranges, f -> id between f.b and f.e)) > 0) as part1
     , aggregate( array_sort(any_value(ranges))
                , array()::array<struct<b:bigint,e:bigint>>
                , (acc, x) -> case
                                when x.b > coalesce(element_at(acc.e, -1), 0)
                                then array_append( acc, x )
                                else array_append( slice(acc, 1, array_size(acc) - 1)
                                                 , struct(element_at(acc, -1).b, greatest(x.e, element_at(acc, -1).e) as e) )
                              end
                , acc -> aggregate(acc, 0l, (agg, y) -> agg + y.e - y.b + 1l)
                ) as part2
  from cte_data;
